# HealthAIBot

In [7]:
# export env variables before running this block
import os

class KEYS:
    OPENAI_API_KEY = os.getenv("OPENAI_API_KEY", "your-openai-api-key")
    TAVILY_API_KEY = os.getenv("TAVILY_API_KEY", "your-tavily-api-key")

In [2]:
# /usr/bin/env python3
# healthAiBot/healthaibot/utils/agent_utils.py
"""
Utility functions for HealthBot agent operations.
"""

from langchain_tavily import TavilySearch


class GraphHelper:
    """
    Helper class for managing graph-related operations.
    """
    def __init__(
        self,
    ) -> None:
        """
        Initialize with the current state.
        """

    def ask_patient(
        self,
        state: dict,
    ) -> dict:
        """
        Prompt the user for a health topic and update the state.
        """
        topic = input("What health topic or medical condition would you like to learn about? ")
        state['topic'] = topic
        print(f"You have chosen to learn about: {state['topic']}")
        return state

    def search_tavily(
        self,
        state: dict,
    ) -> dict:
        """
        Search for relevant information using the Tavily API.
        """
        tavily_tool = TavilySearch()
        query = state['topic'] + " site:nih.gov OR site:mayoclinic.org OR site:webmd.com"
        results = tavily_tool.run(query)
        state['search_results'] = results
        return state

    def summarize_results(
        self,
        state: dict,
    ) -> dict:
        """
        Summarize the search results using the LLM.
        """
        llm = state['llm']
        focus = state.get('focus', None)
        base_prompt = (
            "Summarize the following medical information for a patient in simple, friendly language. "
            "Use a friendly tone, clear explanations, and actionable advice. "
            "If the user requests a specific focus (e.g., symptoms, treatment, prevention), focus on that aspect. "
        )
        if focus:
            base_prompt += f"Focus on: {focus}. "
        prompt = base_prompt + f"\n{state['search_results']}"
        summary = llm.invoke(prompt)
        state['summary'] = summary
        return state

    def present_summary(
        self,
        state: dict,
    ) -> dict:
        """
        Present the summarized information to the user.
        """
        print("\nHere is a summary of what you asked about:\n")
        print(state['summary'])
        return state

    def comprehension_prompt(
        self,
        state: dict,
    ) -> dict:
        """
        Prompt the user for a comprehension check.
        """
        input("\nPress Enter when you are ready to take a comprehension check.")
        return state

    def create_quiz(
        self,
        state: dict,
    ) -> dict:
        """
        Create a quiz based on the current state.
        """
        llm = state['llm']
        previous_questions = state.get('previous_questions', [])
        prompt = (
            "Create a multiple-choice quiz question that is directly relevant to the following summary. "
            "The question should have exactly 4 distinct answer options labeled a), b), c), and d). "
            "Do NOT reveal the correct answer. "
            "Do NOT repeat previous questions. "
            "Format your response as:\n"
            "Question:\n<your question>\n"
            "a) <option 1>\n"
            "b) <option 2>\n"
            "c) <option 3>\n"
            "d) <option 4>\n"
            f"Summary: {state['summary']}\n"
            f"Previous questions: {previous_questions}"
        )
        quiz_question = llm.invoke(prompt)
        state['quiz_question'] = quiz_question
        return state

    def present_quiz(
        self,
        state: dict,
    ) -> dict:
        """
        Present the quiz question to the user.
        """
        print("\nQuiz Question:\n")
        print(state['quiz_question'])
        return state

    def get_quiz_answer(
        self,
        state: dict,
    ) -> dict:
        """
        Get the user's answer to the quiz question.
        """
        answer = input("\nEnter your answer to the quiz question: ")
        state['quiz_answer'] = answer
        return state

    def grade_quiz(
        self,
        state: dict,
    ) -> dict:
        """
        Grade the user's answer to the quiz question.
        """
        llm = state['llm']
        prompt = (
            "Grade the following answer to the quiz question. "
            "Respond with 'Correct' or 'Incorrect', and provide a brief explanation referencing the summary and the correct answer. "
            "Include citations from the summary if possible.\n"
            f"Summary: {state['summary']}\n"
            f"Question: {state['quiz_question']}\n"
            f"Answer: {state['quiz_answer']}"
        )
        grading = llm.invoke(prompt)
        state['grading'] = grading
        return state

    def present_feedback(
        self,
        state: dict,
    ) -> dict:
        """
        Present the feedback to the user.
        """
        print("\nYour grade and feedback:\n")
        print(state['grading'])
        # Prompt user for next action
        while True:
            next_action = input("Would you like to take another quiz on this topic (enter 'quiz'), or learn about a new topic (enter 'new'), or exit (enter 'exit')? ")
            if next_action.lower() == 'quiz':
                # Optionally, you could loop back to quiz creation and answering
                print("Let's take another quiz on this topic!")
                return 'quiz'
            elif next_action.lower() == 'new':
                print("Let's learn about a new topic!")
                return 'new'
            elif next_action.lower() == 'exit':
                print("Thank you for using HealthBot. Stay healthy!")
                return 'exit'
            else:
                print("Invalid input. Please enter 'quiz', 'new', or 'exit'.")

In [3]:
# /usr/bin/env python3
# healthAiBot/healthaibot/utils/utils.py
"""
Utility functions for HealthBot operations.
"""

from langchain_openai import ChatOpenAI
from langchain_ollama.llms import OllamaLLM as Ollama


class HealthBotUtils:
    """
    Utility functions for HealthBot operations.
    """
    def __init__(
        self,
        llm_type: str,
        model_name: str,
        temperature: float = 0.7,
    ) -> None:
        """
        Initialize the HealthBotUtils class.
        Parameters:
            llm_type: Type of LLM to use ('openai' or 'ollama').
            model_name: Name of the model to use.
            temperature: Sampling temperature for the LLM.
        """
        self.llm_type = llm_type
        self.model_name = model_name
        self.temperature = temperature

    def get_llm(
        self,
    ) -> ChatOpenAI | Ollama:
        """
        Get the LLM instance based on the specified type.
        Returns:
            An instance of ChatOpenAI or Ollama.
        """
        if self.llm_type == "openai":
            return ChatOpenAI(
                model=self.model_name,
                temperature=self.temperature
            )
        elif self.llm_type == "ollama":
            return Ollama(
                model=self.model_name,
                temperature=self.temperature
            )
        else:
            raise ValueError("Unsupported LLM type. Choose 'openai' or 'ollama'.")

    def reset_state(
        self,
        llm: ChatOpenAI | Ollama,
    ) -> dict:
        """
        Reset the state of the HealthBot.
        """
        # Clear previous health information to maintain privacy
        return {
            'topic': None,
            'search_results': None,
            'summary': None,
            'quiz_question': None,
            'quiz_answer': None,
            'grading': None,
            'llm': llm
        }

    def parse_quiz(
        self,
        quiz_text: str,
    ) -> tuple[str, list[str]]:
        """
        Parse the quiz text into a question and options.
        """
        # Simple parser to split question and options
        lines = quiz_text.strip().split('\n')
        question = ""
        options = []
        for line in lines:
            if line.startswith("a)") or line.startswith("b)") or line.startswith("c)") or line.startswith("d)"):
                options.append(line)
            elif line.lower().startswith("question:"):
                question = line[len("Question:"):].strip()
            elif line and not line.startswith("summary:"):
                question += " " + line.strip()
        
        return question, options


In [4]:
# /usr/bin/env python3
"""
healthAiBot graph definition.
"""

from langgraph.graph import StateGraph, END


def build_healthbot_graph() -> None:
    """
    Build the HealthBot graph with nodes and transitions.
    """
    helper = GraphHelper()
    graph = StateGraph(dict)

    graph.add_node("ask_patient", helper.ask_patient)
    graph.add_node("search_tavily", helper.search_tavily)
    graph.add_node("summarize_results", helper.summarize_results)
    graph.add_node("present_summary", helper.present_summary)
    graph.add_node("comprehension_prompt", helper.comprehension_prompt)
    graph.add_node("create_quiz", helper.create_quiz)
    graph.add_node("present_quiz", helper.present_quiz)
    graph.add_node("get_quiz_answer", helper.get_quiz_answer)
    graph.add_node("grade_quiz", helper.grade_quiz)
    graph.add_node("present_feedback", helper.present_feedback)
    graph.add_edge("ask_patient", "search_tavily")
    graph.add_edge("search_tavily", "summarize_results")
    graph.add_edge("summarize_results", "present_summary")
    graph.add_edge("present_summary", "comprehension_prompt")
    graph.add_edge("comprehension_prompt", "create_quiz")
    graph.add_edge("create_quiz", "present_quiz")
    graph.add_edge("present_quiz", "get_quiz_answer")
    graph.add_edge("get_quiz_answer", "grade_quiz")
    graph.add_edge("grade_quiz", "present_feedback")
    graph.add_edge("present_feedback", END)
    graph.set_entry_point("ask_patient")


In [5]:
# /usr/bin/env python3
"""
HealthBot Command Line Interface (CLI)
This script provides a command-line interface for interacting with the HealthBot application.
Users can specify various parameters such as the LLM backend, model name, and temperature.
"""

#import argparse


def main():
    """
    Main function to run the HealthBot CLI.
    """
    '''
    parser = argparse.ArgumentParser(description="HealthBot CLI")
    parser.add_argument(
        '--llm_type',
        choices=['openai', 'ollama'],
        default='ollama',
        help='Choose LLM backend: openai or ollama'
    )
    parser.add_argument(
        '--model_name',
        type=str,
        default='gemma3:1b',
        help='Model name for LLM'
    )
    parser.add_argument(
        '--temperature',
        type=float,
        default=0.3,
        help='Temperature for LLM'
    )
    # Add more arguments as needed
    args = parser.parse_args()
    '''

    llm_type = "ollama"
    model_name = "gemma3:1b"
    temperature = 0.3
    build_healthbot_graph()

    healthbot = HealthBotUtils(
        llm_type=llm_type,
        model_name=model_name,
        temperature=temperature,
    )

    llm = healthbot.get_llm()

    graphhelper = GraphHelper()

    print("Welcome to HealthBot!")
    while True:
        state = healthbot.reset_state(llm)
        state['topic'] = input("What health topic or medical condition would you like to learn about? ")
        print(f"You have chosen to learn about: {state['topic']}")

        # Search and summarize
        state = graphhelper.search_tavily(state)
        # Ask for focus after summary
        focus = input("Do you want to focus on a specific aspect (e.g., symptoms, treatment, prevention)? If yes, enter it, otherwise press Enter: ")
        if focus.strip():
            state['focus'] = focus.strip()
        state = graphhelper.summarize_results(state)
        graphhelper.present_summary(state)
        graphhelper.comprehension_prompt(state)

        # Track previous questions for this topic
        state['previous_questions'] = []
        quiz_active = True
        while quiz_active:
            # Create and present quiz
            state = graphhelper.create_quiz(state)
            question, options = healthbot.parse_quiz(state['quiz_question'])
            print("\nQuiz Question:\n" + question)
            for opt in options:
                print(opt)
            state['previous_questions'].append(question)
            state = graphhelper.get_quiz_answer(state)
            state = graphhelper.grade_quiz(state)
            graphhelper.present_feedback(state)

            next_action = input("Would you like to take another quiz on this topic (enter 'quiz'), learn about a new topic (enter 'new'), or exit (enter 'exit')? ")
            if next_action.lower() == 'quiz':
                continue  # Stay in quiz loop, do not prompt again
            elif next_action.lower() == 'new':
                quiz_active = False  # Break quiz loop, go to new topic
            elif next_action.lower() == 'exit':
                print("Thank you for using HealthBot. Stay healthy!")
                return
            else:
                print("Invalid input. Please enter 'quiz', 'new', or 'exit'.")

In [6]:
main()

Welcome to HealthBot!


What health topic or medical condition would you like to learn about?  common cold


You have chosen to learn about: common cold


Do you want to focus on a specific aspect (e.g., symptoms, treatment, prevention)? If yes, enter it, otherwise press Enter:  treatment



Here is a summary of what you asked about:

Okay, let’s talk about the common cold! It’s a really common illness, and it can be a bit of a bummer, but it’s usually pretty easy to manage.

Basically, the common cold is caused by a tiny virus – usually rhinoviruses – that can irritate your nose and throat. It’s more common in kids and babies because they haven’t been exposed to as many viruses as adults.

**What you can do to feel better right now:**

*   **Rest:** Give your body a chance to recover.
*   **Stay Hydrated:** Drink plenty of fluids like water, tea, or broth.
*   **Soothe a Sore Throat:** Gargle with warm salt water (1/4 teaspoon salt in 8 ounces of water) several times a day.
*   **Nasal Rinse:** A saline nasal rinse can help clear congestion.

**Important Note:**  If your symptoms are getting worse, or you have a fever, cough, or chest pain, it’s a good idea to see a doctor.

**Do you want me to focus on anything specific, like:**

*   **Symptoms?** (e.g., what to expect?

...


Your grade and feedback:

Correct.

**Explanation:** The summary explicitly states that the common cold is caused by a “tiny virus – usually rhinoviruses” that irritates your nose and throat. Option (b) directly reflects this information.


Would you like to take another quiz on this topic (enter 'quiz'), or learn about a new topic (enter 'new'), or exit (enter 'exit')?  exit


Thank you for using HealthBot. Stay healthy!


Would you like to take another quiz on this topic (enter 'quiz'), learn about a new topic (enter 'new'), or exit (enter 'exit')?  exit


Thank you for using HealthBot. Stay healthy!
